In [113]:
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [116]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [117]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]


In [119]:
# 30 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,1,1)
end_datetime = datetime(2025,2,22)
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

,time,open,high,low,close
0,2024-12-31 23:00:00,1.03586,1.03607,1.03586,1.03604
1,2024-12-31 23:01:00,1.03604,1.03609,1.03604,1.03609
2,2024-12-31 23:02:00,1.03609,1.03610,1.03602,1.03603
3,2024-12-31 23:03:00,1.03603,1.03603,1.03592,1.03592
4,2024-12-31 23:04:00,1.03592,1.03597,1.03591,1.03595
...,...,...,...,...,...
53204,2025-02-21 22:56:00,1.04604,1.04606,1.04575,1.04590
53205,2025-02-21 22:57:00,1.04590,1.04607,1.04590,1.04601
53206,2025-02-21 22:58:00,1.04601,1.04612,1.04600,1.04600
53207,2025-02-21 22:59:00,1.04600,1.04611,1.04599,1.04608


In [120]:
# hour4 df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_H4
start_datetime = datetime(2025,1,1)
end_datetime = datetime(2025,2,22)
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df

,time,open,high,low,close
0,2025-01-02 00:00:00,1.03515,1.03609,1.03443,1.03579
1,2025-01-02 04:00:00,1.03579,1.03756,1.03578,1.03638
2,2025-01-02 08:00:00,1.03638,1.03725,1.03464,1.03517
3,2025-01-02 12:00:00,1.03517,1.03569,1.03115,1.03134
4,2025-01-02 16:00:00,1.03134,1.03239,1.02243,1.02650
...,...,...,...,...,...
217,2025-02-21 04:00:00,1.04870,1.05050,1.04865,1.04929
218,2025-02-21 08:00:00,1.04929,1.04997,1.04668,1.04693
219,2025-02-21 12:00:00,1.04693,1.04779,1.04593,1.04752
220,2025-02-21 16:00:00,1.04752,1.04927,1.04494,1.04632


In [121]:
# daily df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_D1
start_datetime = datetime(2025,1,1)
end_datetime = datetime(2025,2,22)
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df

,time,open,high,low,close
0,2025-01-02,1.03515,1.03756,1.02243,1.02670
1,2025-01-03,1.02572,1.03099,1.02529,1.03080
2,2025-01-06,1.03021,1.04369,1.02955,1.03897
3,2025-01-07,1.03896,1.04345,1.03396,1.03398
4,2025-01-08,1.03401,1.03577,1.02732,1.03191
5,2025-01-09,1.03181,1.03216,1.02836,1.02994
6,2025-01-10,1.02989,1.03119,1.02134,1.02424
7,2025-01-13,1.02236,1.02499,1.01773,1.02424
8,2025-01-14,1.02443,1.03084,1.02379,1.03077
9,2025-01-15,1.03047,1.03544,1.02593,1.02902


In [155]:
# start b2 strategy
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci retracement
daily_df["pivot"] = ((daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3).shift(1)
daily_df["prange"] = (daily_df["high"] - daily_df["low"]).shift(1)
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df[["time","S2","R2"]]

,time,S2,R2
0,2025-01-02,NaN,NaN
1,2025-01-03,1.019546,1.038247
2,2025-01-06,1.025504,1.032549
3,2025-01-07,1.028665,1.046142
4,2025-01-08,1.031265,1.042995
5,2025-01-09,1.026445,1.036889
6,2025-01-10,1.027805,1.032502
7,2025-01-13,1.019503,1.031677
8,2025-01-14,1.017833,1.026807
9,2025-01-15,1.024110,1.032824


In [156]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
#ohlc4
hourly_df['ohlc4'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4

# sma 1
hourly_df['sma_1'] = hourly_df['ohlc4'].rolling(furst_sma_period).mean()
hourly_df['prev_sma_1'] = hourly_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
hourly_df['sma_14'] = hourly_df['ohlc4'].rolling(second_sma_period).mean()
hourly_df

,time,open,high,low,close,sma_1,prev_sma_1,sma_14,ohlc4
0,2025-01-02 00:00:00,1.03515,1.03609,1.03443,1.03579,1.035365,NaN,NaN,1.035365
1,2025-01-02 04:00:00,1.03579,1.03756,1.03578,1.03638,1.036377,1.035365,NaN,1.036377
2,2025-01-02 08:00:00,1.03638,1.03725,1.03464,1.03517,1.035860,1.036377,NaN,1.035860
3,2025-01-02 12:00:00,1.03517,1.03569,1.03115,1.03134,1.033338,1.035860,NaN,1.033338
4,2025-01-02 16:00:00,1.03134,1.03239,1.02243,1.02650,1.028165,1.033338,NaN,1.028165
...,...,...,...,...,...,...,...,...,...
217,2025-02-21 04:00:00,1.04870,1.05050,1.04865,1.04929,1.049285,1.049448,1.044896,1.049285
218,2025-02-21 08:00:00,1.04929,1.04997,1.04668,1.04693,1.048218,1.049285,1.045158,1.048218
219,2025-02-21 12:00:00,1.04693,1.04779,1.04593,1.04752,1.047042,1.048218,1.045297,1.047042
220,2025-02-21 16:00:00,1.04752,1.04927,1.04494,1.04632,1.047012,1.047042,1.045485,1.047012


In [157]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
#ohlc4
daily_df['ohlc4'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
# sma 11
daily_df['sma_11'] = daily_df['ohlc4'].rolling(thurd_sma_period).mean()
daily_df['prev_sma_11'] = daily_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
daily_df['sma_114'] = daily_df['ohlc4'].rolling(fourth_sma_period).mean()
daily_df[["time","sma_11","sma_114","prev_sma_11"]]

,time,sma_11,sma_114,prev_sma_11
0,2025-01-02,1.030460,NaN,NaN
1,2025-01-03,1.028200,NaN,1.030460
2,2025-01-06,1.035605,NaN,1.028200
3,2025-01-07,1.037588,NaN,1.035605
4,2025-01-08,1.032253,NaN,1.037588
5,2025-01-09,1.030567,NaN,1.032253
6,2025-01-10,1.026665,NaN,1.030567
7,2025-01-13,1.022330,NaN,1.026665
8,2025-01-14,1.027458,NaN,1.022330
9,2025-01-15,1.030215,NaN,1.027458


In [ ]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('17:00:00', '%H:%M:%S').time()
end_trading_section = datetime.strptime('23:45:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]
# open one trade a day
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
# Close all open positions at 11:45 PM
open_positions = daily_df[daily_df['time'].dt.time == end_trading_section]

# function to trading
daily_df["sell_stope_loss"] = daily_df["high"] >= daily_df["R2"] + 0.0001*40
daily_df["buy_stope_loss"] = daily_df["low"] <= daily_df["S2"] - 0.0001*40
daily_df['sell_profit'] = daily_df['low'] <= daily_df["R1"] - daily_df['R3'] + daily_df["R1"] - 0.0001*40
daily_df['buy_profit'] = daily_df['high'] >= daily_df["S1"] - daily_df["S3"] + daily_df["S1"] + 0.0001*40
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,prev_sma_11,sma_114,sell_stope_loss,bearish_pivot,buy_stope_loss,bullish_pivot,sell_profit,buy_profit,signal,ohlc4
0,2025-01-02,1.03515,1.03756,1.02243,1.02670,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,nothing,1.030460
1,2025-01-03,1.02572,1.03099,1.02529,1.03080,1.028897,0.01513,1.034676,1.038247,1.044027,...,1.030460,NaN,False,False,False,False,False,False,nothing,1.028200
2,2025-01-06,1.03021,1.04369,1.02955,1.03897,1.029027,0.00570,1.031204,1.032549,1.034727,...,1.028200,NaN,True,True,False,False,False,True,nothing,1.035605
3,2025-01-07,1.03896,1.04345,1.03396,1.03398,1.037403,0.01414,1.042805,1.046142,1.051543,...,1.035605,NaN,False,False,False,False,False,False,nothing,1.037588
4,2025-01-08,1.03401,1.03577,1.02732,1.03191,1.037130,0.00949,1.040755,1.042995,1.046620,...,1.037588,NaN,False,False,False,True,True,False,nothing,1.032253
5,2025-01-09,1.03181,1.03216,1.02836,1.02994,1.031667,0.00845,1.034895,1.036889,1.040117,...,1.032253,NaN,False,False,False,False,False,False,nothing,1.030567
6,2025-01-10,1.02989,1.03119,1.02134,1.02424,1.030153,0.00380,1.031605,1.032502,1.033953,...,1.030567,NaN,False,False,True,True,True,False,nothing,1.026665
7,2025-01-13,1.02236,1.02499,1.01773,1.02424,1.025590,0.00985,1.029353,1.031677,1.035440,...,1.026665,NaN,False,False,False,True,True,False,nothing,1.022330
8,2025-01-14,1.02443,1.03084,1.02379,1.03077,1.022320,0.00726,1.025093,1.026807,1.029580,...,1.022330,NaN,True,True,False,False,False,True,nothing,1.027458
9,2025-01-15,1.03047,1.03544,1.02593,1.02902,1.028467,0.00705,1.031160,1.032824,1.035517,...,1.027458,NaN,False,True,False,False,False,True,nothing,1.030215


In [201]:
def entring_stoploss_profit(row, ohlc_df) :
    if row["prev_sma_11"] > row['sma_114'] and row["low"] <= row["S2"] :
        return "buy"
    elif row['prev_sma_11'] < row['sma_114'] and row["high"] >= row["R2"] :
        return "sell"  
    else :
        return "nothing"

daily_df['signal'] = daily_df.apply(entring_stoploss_profit, axis=1, ohlc_df = ohlc_df)
daily_df[["signal","time"]]

,signal,time
0,nothing,2025-01-02
1,nothing,2025-01-03
2,nothing,2025-01-06
3,nothing,2025-01-07
4,nothing,2025-01-08
5,nothing,2025-01-09
6,nothing,2025-01-10
7,nothing,2025-01-13
8,nothing,2025-01-14
9,nothing,2025-01-15


In [ ]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    if 'signal' not in data:
        print("Warning: 'signal' column is missing!")
        return
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy', sl= data["low"] <= (data["R1"] - data["S2"]) /2 - data["S1"] , tp= data["high"] >= data["R1"])
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell', sl= data["high"] >= (data["R2"] - data["S1"]) /2 + data["R2"] ,tp= data["low"] <= data["S1"])
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if (trade['order_type'] == 'buy' and ((data["low"] <= (data["R1"] - data["S2"]) /2 - data["S1"]) 
                                              or (data["high"] >= data["R1"]) or (data['time'].time() == pd.to_datetime('23:45:00').time()))):
            orders.close_trade(trade)
        elif (trade['order_type'] == 'sell' and ((data["high"] >= (data["R2"] - data["S1"]) /2 + data["R2"]) 
                                                 or (data["low"] <= data["S1"]) or (data['time'].time() == pd.to_datetime('23:45:00').time()))):
            orders.close_trade(trade)

In [219]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(daily_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,100000,2025-01-23 00:00:00,1.04083,2025-01-24 00:00:00,1.0415,False,False,{},67.0,-7.0,60.0,60.0,10060.0
1,closed,EURUSD,buy,100000,2025-01-28 00:00:00,1.04915,2025-01-30 00:00:00,1.04207,False,False,{},-708.0,-7.0,-715.0,-655.0,9345.0
2,closed,EURUSD,buy,100000,2025-01-31 00:00:00,1.03882,2025-02-04 00:00:00,1.03436,False,False,{},-446.0,-7.0,-453.0,-1108.0,8892.0
3,closed,EURUSD,sell,100000,2025-02-05 00:00:00,1.03785,2025-02-06 00:00:00,1.0402,False,False,{},-235.0,-7.0,-242.0,-1350.0,8650.0
4,closed,EURUSD,sell,100000,2025-02-11 00:00:00,1.03065,2025-02-11 00:00:00,True,True,False,{},3065.0,-7.0,3058.0,1708.0,11708.0
5,closed,EURUSD,sell,100000,2025-02-12 00:00:00,1.03609,2025-02-18 00:00:00,1.04836,False,False,{},-1227.0,-7.0,-1234.0,474.0,10474.0
6,closed,EURUSD,buy,100000,2025-02-19 00:00:00,1.04454,2025-02-20 00:00:00,1.04225,False,False,{},-229.0,-7.0,-236.0,238.0,10238.0


In [ ]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                        open = ohlc_df['open'],
                                        high = ohlc_df['high'],
                                        low = ohlc_df['low'],
                                        close = ohlc_df['close'])])

# fig.update_layout(height=600, xaxis_rangeslider_visible=False)
# 📌 Add Pivot Points as Horizontal Lines
pivot_levels = ["R1", "R2", "S1", "S2"]
colors = {'R1': 'red', 'R2': 'red', 'S1': 'green', 'S2': 'green'}


for _, row in daily_df.iterrows():
    for level in pivot_levels:
        fig.add_trace(go.Scatter(
            x=[row['time'], row['time'] + pd.Timedelta(days=1)],  # Extend for one day
            y=[row[level], row[level]],
            mode='lines',
            name=f"{level} {row['time']}",
            line=dict(color=colors[level], width=1.5, dash='dot')
        ))

# 📌 Customize Layout
fig.update_layout(
    title="Pivot Points on Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': {'bdata': ('NIXOa+yS8D+NeohGd5DwP+KvyRr1kP' ... 'vIB73wP7D+z2G+vPA/XMmOjUC88D8='),
                        'dtype': 'f8'},
              'high': {'bdata': ('ayv2l92T8D8W9rTDX5PwP4xK6gQ0kf' ... 'nXTL7wP3bgnBGlvfA/6LzGLlG98D8='),
                       'dtype': 'f8'},
              'low': {'bdata': ('w9hCkIOS8D+NeohGd5DwP8nIWdjTjv' ... 'reyLzwPwisHFpku/A/zqW4quy78D8='),
                      'dtype': 'f8'},
              'open': {'bdata': ('wsBz7+GS8D80hc5r7JLwPzlFR3L5j/' ... 'YuUb3wP8xdS8gHvfA/sP7PYb688D8='),
                       'dtype': 'f8'},
              'type': 'candlestick',
              'x': array(['2024-12-31T23:00:00.000000000', '2024-12-31T23:30:00.000000000',
                          '2025-01-02T00:00:00.000000000', ..., '2025-02-21T22:00:00.000000000',
                          '2025-02-21T22:30:00.000000000', '2025-02-21T23:00:00.000000000'],
                         shape=(1777,), dtype='datetime64[ns]')},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R1 2025-01-02 00:00:00',
              'type': 'scatter',
              'x': [2025-01-02 00:00:00, 2025-01-03 00:00:00],
              'y': [1.0346763266666665, 1.0346763266666665]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R2 2025-01-02 00:00:00',
              'type': 'scatter',
              'x': [2025-01-02 00:00:00, 2025-01-03 00:00:00],
              'y': [1.0382470066666667, 1.0382470066666667]},
             {'line': {'color': 'green', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'S1 2025-01-02 00:00:00',
              'type': 'scatter',
              'x': [2025-01-02 00:00:00, 2025-01-03 00:00:00],
              'y': [1.0231170066666666, 1.0231170066666666]},
             {'line': {'color': 'green', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'S2 2025-01-02 00:00:00',
              'type': 'scatter',
              'x': [2025-01-02 00:00:00, 2025-01-03 00:00:00],
              'y': [1.0195463266666664, 1.0195463266666664]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R1 2025-01-03 00:00:00',
              'type': 'scatter',
              'x': [2025-01-03 00:00:00, 2025-01-04 00:00:00],
              'y': [1.0312040666666669, 1.0312040666666669]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R2 2025-01-03 00:00:00',
              'type': 'scatter',
              'x': [2025-01-03 00:00:00, 2025-01-04 00:00:00],
              'y': [1.0325492666666667, 1.0325492666666667]},
             {'line': {'color': 'green', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'S1 2025-01-03 00:00:00',
              'type': 'scatter',
              'x': [2025-01-03 00:00:00, 2025-01-04 00:00:00],
              'y': [1.0268492666666666, 1.0268492666666666]},
             {'line': {'color': 'green', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'S2 2025-01-03 00:00:00',
              'type': 'scatter',
              'x': [2025-01-03 00:00:00, 2025-01-04 00:00:00],
              'y': [1.0255040666666668, 1.0255040666666668]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R1 2025-01-06 00:00:00',
              'type': 'scatter',
              'x': [2025-01-06 00:00:00, 2025-01-07 00:00:00],
              'y': [1.0428048133333334, 1.0428048133333334]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R2 2025-01-06 00:00:00',
              'type': 'scatter',
              'x': [2025-01-06 00:00:00, 2025-01-07 00:00:00],
              'y': [1.04